In [1]:
import os
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sns



In [20]:
matched_ratings = pd.read_csv("src/data/data_matched_beer_data/ratings.csv", header=[0, 1])
matched_ratings.columns = ['_'.join(col).strip() for col in matched_ratings.columns.values]
matched_ratings['ba_date'] = pd.to_datetime(matched_ratings['ba_date'], unit='s')
matched_ratings['rb_date'] = pd.to_datetime(matched_ratings['rb_date'], unit='s')
matched_ratings['ba_user_name'] = matched_ratings['ba_user_name'].str.lower()
matched_ratings['rb_user_name'] = matched_ratings['rb_user_name'].str.lower()

# ba_beer_first_review = matched_ratings.groupby('ba_beer_id').agg(
#     ba_first_review_date=('ba_date', 'min')
# ).reset_index()
# first_reviews = df.groupby(['beer_id', 'site_id']).agg(
#     first_review_date=('review_date', 'min')
# ).reset_index()
first_reviews = matched_ratings.groupby('ba_beer_id').agg({
    'ba_date': 'min',
    'rb_date': 'min'
}).reset_index()


first_reviews.dropna(subset=['ba_date', 'rb_date'], inplace=True)

# Calculate the time difference in days between first reviews on both sites
first_reviews['time_diff'] = (first_reviews['ba_date'] - first_reviews['rb_date']).abs().dt.days

# Define the time span threshold (e.g., 30 days)
threshold = 30

# Identify beers where the time difference is within the threshold
newcomer_beers = first_reviews[first_reviews['time_diff'] <= threshold]

# Output the list of newcomer beers
print("Beers with first reviews on both sites within 30 days:")
print(newcomer_beers[['ba_beer_id', 'ba_date', 'rb_date', 'time_diff']])

Beers with first reviews on both sites within 30 days:
      ba_beer_id             ba_date             rb_date  time_diff
0             12 2003-04-09 10:00:00 2003-04-09 10:00:00          0
3             26 2003-09-23 10:00:00 2003-09-22 10:00:00          1
5             58 2003-04-09 10:00:00 2003-04-08 10:00:00          1
8            120 2012-05-01 10:00:00 2012-04-04 10:00:00         27
9            125 2004-07-28 10:00:00 2004-07-28 10:00:00          0
...          ...                 ...                 ...        ...
9020      292548 2017-07-20 10:00:00 2017-07-19 10:00:00          1
9021      292645 2017-07-20 10:00:00 2017-07-20 10:00:00          0
9022      292835 2017-07-21 10:00:00 2017-07-20 10:00:00          1
9023      292882 2017-07-23 10:00:00 2017-07-23 10:00:00          0
9024      293187 2017-07-23 10:00:00 2017-07-24 10:00:00          1

[6980 rows x 4 columns]


In [ ]:
# Load the matched ratings dataset
matched_ratings = pd.read_csv("src/data/data_matched_beer_data/ratings.csv", header=[0, 1])
matched_ratings.columns = ['_'.join(col).strip() for col in matched_ratings.columns.values]

# Convert date columns to datetime
matched_ratings['ba_date'] = pd.to_datetime(matched_ratings['ba_date'], unit='s')
matched_ratings['rb_date'] = pd.to_datetime(matched_ratings['rb_date'], unit='s')

# Convert usernames to lowercase
matched_ratings['ba_user_name'] = matched_ratings['ba_user_name'].str.lower()
matched_ratings['rb_user_name'] = matched_ratings['rb_user_name'].str.lower()

# Find the first review date and corresponding user for each beer on BeerAdvocate
ba_first_reviews = matched_ratings.groupby('ba_beer_id').agg({'ba_date': 'min'}).reset_index()
ba_first_reviews = ba_first_reviews.merge(
    matched_ratings[['ba_beer_id', 'ba_date', 'ba_user_name']],
    on=['ba_beer_id', 'ba_date'],
    how='left'
)

# Find the first review date and corresponding user for each beer on RateBeer
rb_first_reviews = matched_ratings.groupby('ba_beer_id').agg({'rb_date': 'min'}).reset_index()
rb_first_reviews = rb_first_reviews.merge(
    matched_ratings[['ba_beer_id', 'rb_date', 'rb_user_name']],
    on=['ba_beer_id', 'rb_date'],
    how='left'
)

# Merge the first reviews from both sites on 'ba_beer_id'
first_reviews = pd.merge(ba_first_reviews, rb_first_reviews, on='ba_beer_id', how='inner')

# Ensure that the first reviewers are not the same person
first_reviews = first_reviews[first_reviews['ba_user_name'] != first_reviews['rb_user_name']]

# Drop any rows with missing dates
first_reviews.dropna(subset=['ba_date', 'rb_date'], inplace=True)

# Calculate the time difference in days between first reviews on both sites
first_reviews['time_diff'] = (first_reviews['ba_date'] - first_reviews['rb_date']).abs().dt.days

# Define the time span thresholds
threshold = 150
security = 1

# Identify beers where the time difference is within the thresholds
newcomer_beers = first_reviews[
    (first_reviews['time_diff'] >= security) & (first_reviews['time_diff'] <= threshold)
]

# Output the list of newcomer beers
print("Beers with first reviews on both sites within the specified time frame:")
print(newcomer_beers[['ba_beer_id', 'ba_date', 'rb_date', 'time_diff']])
newcomer_beers.head()



Beers with first reviews on both sites within the specified time frame:
      ba_beer_id             ba_date             rb_date  time_diff
5             58 2003-04-09 10:00:00 2003-04-08 10:00:00          1
10           145 2002-10-07 10:00:00 2002-07-03 10:00:00         96
11           146 2003-01-05 11:00:00 2003-04-27 10:00:00        111
29           467 2003-02-08 11:00:00 2002-09-19 10:00:00        142
30           469 2003-03-07 11:00:00 2003-02-22 11:00:00         13
...          ...                 ...                 ...        ...
7993      243179 2016-08-21 10:00:00 2016-09-01 10:00:00         11
8181      250599 2017-01-10 11:00:00 2017-01-08 11:00:00          2
8305      255794 2016-11-10 11:00:00 2016-06-25 10:00:00        138
8596      267178 2017-02-24 11:00:00 2017-02-26 11:00:00          2
8700      271719 2017-03-13 11:00:00 2017-03-11 11:00:00          2

[400 rows x 4 columns]


,ba_beer_id,ba_date,ba_user_name,rb_date,rb_user_name,time_diff
5,58,2003-04-09 10:00:00,goz,2003-04-08 10:00:00,beerguy101,1
10,145,2002-10-07 10:00:00,dogbrick,2002-07-03 10:00:00,gusler,96
11,146,2003-01-05 11:00:00,aaronh,2003-04-27 10:00:00,goz,111
29,467,2003-02-08 11:00:00,gusler,2002-09-19 10:00:00,beerguy101,142
30,469,2003-03-07 11:00:00,gusler,2003-02-22 11:00:00,beerguy101,13


In [65]:
# Load the matched ratings dataset
matched_ratings = pd.read_csv("src/data/data_matched_beer_data/ratings.csv", header=[0, 1])
matched_ratings.columns = ['_'.join(col).strip() for col in matched_ratings.columns.values]

# Convert date columns to datetime
matched_ratings['ba_date'] = pd.to_datetime(matched_ratings['ba_date'], unit='s')
matched_ratings['rb_date'] = pd.to_datetime(matched_ratings['rb_date'], unit='s')

# Convert usernames to lowercase
matched_ratings['ba_user_name'] = matched_ratings['ba_user_name'].str.lower()
matched_ratings['rb_user_name'] = matched_ratings['rb_user_name'].str.lower()

# Find the first review date and corresponding user for each beer on BeerAdvocate
ba_first_reviews = matched_ratings.groupby('rb_beer_id').agg({'ba_date': 'min'}).reset_index()
ba_first_reviews = ba_first_reviews.merge(
    matched_ratings[['rb_beer_id', 'ba_date', 'ba_user_name']],
    on=['rb_beer_id', 'ba_date'],
    how='left'
)

# Find the first review date and corresponding user for each beer on RateBeer
rb_first_reviews = matched_ratings.groupby('rb_beer_id').agg({'rb_date': 'min'}).reset_index()
rb_first_reviews = rb_first_reviews.merge(
    matched_ratings[['rb_beer_id', 'rb_date', 'rb_user_name']],
    on=['rb_beer_id', 'rb_date'],
    how='left'
)

# Merge the first reviews from both sites on 'ba_beer_id'
first_reviews = pd.merge(ba_first_reviews, rb_first_reviews, on='rb_beer_id', how='inner')

# Ensure that the first reviewers are not the same person
first_reviews = first_reviews[first_reviews['ba_user_name'] != first_reviews['rb_user_name']]

# Drop any rows with missing dates
first_reviews.dropna(subset=['ba_date', 'rb_date'], inplace=True)

# Calculate the time difference in days between first reviews on both sites
first_reviews['time_diff'] = (first_reviews['ba_date'] - first_reviews['rb_date']).abs().dt.days

# Define the time span thresholds
threshold = 150
security = 1

# Identify beers where the time difference is within the thresholds
newcomer_beers = first_reviews[
    (first_reviews['time_diff'] >= security) & (first_reviews['time_diff'] <= threshold)
]

# Output the list of newcomer beers
print("Beers with first reviews on both sites within the specified time frame:")
print(newcomer_beers[['rb_beer_id', 'ba_date', 'rb_date', 'time_diff']])
newcomer_beers.head()


Beers with first reviews on both sites within the specified time frame:
      rb_beer_id             ba_date             rb_date  time_diff
3             15 2003-03-03 11:00:00 2002-10-10 10:00:00        144
6            109 2003-03-07 11:00:00 2003-02-22 11:00:00         13
7            111 2003-02-08 11:00:00 2002-09-19 10:00:00        142
8            178 2002-12-13 11:00:00 2002-10-04 10:00:00         70
17           529 2003-01-05 11:00:00 2003-04-27 10:00:00        111
...          ...                 ...                 ...        ...
7938      426431 2016-10-23 10:00:00 2016-10-16 10:00:00          7
7974      429053 2016-11-10 11:00:00 2016-06-25 10:00:00        138
8046      433930 2017-01-10 11:00:00 2017-01-08 11:00:00          2
8641      484975 2017-02-24 11:00:00 2017-02-26 11:00:00          2
8751      493732 2017-03-13 11:00:00 2017-03-11 11:00:00          2

[400 rows x 4 columns]


,rb_beer_id,ba_date,ba_user_name,rb_date,rb_user_name,time_diff
3,15,2003-03-03 11:00:00,gusler,2002-10-10 10:00:00,beerguy101,144
6,109,2003-03-07 11:00:00,gusler,2003-02-22 11:00:00,beerguy101,13
7,111,2003-02-08 11:00:00,gusler,2002-09-19 10:00:00,beerguy101,142
8,178,2002-12-13 11:00:00,maxpower,2002-10-04 10:00:00,beerguy101,70
17,529,2003-01-05 11:00:00,aaronh,2003-04-27 10:00:00,goz,111


In [ ]:
m